In [1]:
import re
import requests
import pandas as pd
import numpy as np
import pymongo
import json
from nltk.corpus import stopwords
from spacy.en import English
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.corpus import stopwords
from spacy.en import STOP_WORDS
from bs4 import BeautifulSoup
nlp = English()
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [17]:
client = pymongo.MongoClient('54.201.199.246', 27016)

wiki_db = client.wikipedia

wiki_col = wiki_db.my_collection

wiki_col.count()

1636

In [18]:
cursor = wiki_col.find()

In [20]:
# next(cursor)['content']

In [21]:
machine_df = pd.DataFrame(list(cursor))

In [22]:
stop = set(stopwords.words('english'))

In [23]:
def cleaner(text):
    text = text.lower()
    text = re.sub('<.{0,3}>',' ',text)
    text = re.sub('{.*\.*}',' ',text)
    text = re.sub('{.*\+.*}',' ',text)
    text = re.sub('[^a-z]',' ',text)
    text = ' '.join( [w for w in text.split() if len(w)>1] )
    text = re.sub('aa','a',text)
    text = re.sub('[0-9]','',text)
    text = re.sub('\s+',' ',text)
    text = ' '.join([i.lemma_ for i in nlp(text) 
                     if i.orth_ not in STOP_WORDS])
    return text

In [ ]:
def cleaner(text):
    text = re.sub('&#39;','',text).lower()
    text = re.sub('<br />','',text)
    text = re.sub('<.*>.*</.*>','', text)
    text = re.sub('\\ufeff', '', text)
    text = re.sub('[\d]','',text)
    text = re.sub('[^a-z ]','',text)
    text = ' '.join(i.lemma_ for i in nlp(text)
                    if i.orth_ not in STOP_WORDS)
    text = ' '.join(text.split())
    return text

In [20]:
cleaner(machine_df['content'][0]['*'])

'datum set dataset machine learn research cite peer review peer review academic journal publication dataset integral field machine learn major advance field result advance learn algorithm deep learning computer hardware intuitively availability high quality training dataset ref high quality label training dataset supervise learning supervise semi supervise learn semi supervise machine learn algorithm usually difficult expensive produce large time need label datum need label high quality dataset unsupervised learning unsupervised learning difficult costly produce weiss gary foster provost learn training datum costly effect class distribution tree induction journal artificial intelligence research ref turney peter type cost inductive concept learn ref abney steven semisupervis learn computational linguistic crc press ref liobait indr et al active learn evolve stream datum machine learning knowledge discovery database springer berlin heidelberg ref list aggregate high quality dataset show

In [24]:
fixed_content = []
for i, item in enumerate(machine_df['content']):
    content = machine_df['content'][i]['*']
    fixed_content.append(content)

In [25]:
machine_df['content'] = fixed_content

In [26]:
machine_df.head()

,_id,article,category,content,page_id
0,5a11baefc7782d026b5037f7,List of datasets for machine learning research,Machine learning,{{Use dmy dates|date=September 2017}}\n{{machi...,49082762
1,5a11baefc7782d026b5037f8,Machine learning,Machine learning,{{for|the journal|Machine Learning (journal)}}...,233488
2,5a11baf0c7782d026b5037f9,Outline of machine learning,Machine learning,<!--... Attention: THIS IS AN OUTLINE\n\n ...,53587467
3,5a11baf0c7782d026b5037fa,Accuracy paradox,Machine learning,The '''accuracy paradox''' for [[predictive an...,3771060
4,5a11baf0c7782d026b5037fb,Action model learning,Machine learning,{{Machine learning bar}}\n\n'''Action model le...,43808044


In [27]:
machine_df['clean_text'] = machine_df['content'].apply(cleaner)

In [34]:
machine_df['clean_text'][0:15]

0     datum set dataset machine learn research cite ...
1     machine learning field computer science give c...
2     attention outline set outline list portal cont...
3     accuracy paradox predictive analytic state pre...
4     action model learning abbreviated action learn...
5     active learning special case semi supervise le...
6     adversarial machine learning research field li...
7     infobox artist aiva nationality luxembourgish ...
8     multiple issue aixi theoretical mathematical f...
9     algorithm selection call instance algorithm se...
10    algorithmic bias occur datum teach machine lea...
11    algorithmic inference gather new development s...
12    file alphago logo reverse svg alphago logo thu...
13    alphago zero version deepmind software alphago...
14    apprenticeship learning apprenticeship inverse...
Name: clean_text, dtype: object